- Perceptrón simple
- Funciones de activación (ReLU, Sigmoid, Tanh, Softmax)
- Propagación hacia adelante (forward)
- Retropropagación (backpropagation)
- Inicialización de pesos (Xavier, He, etc.)

In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Preprocesamiento del dataset original
df = pd.read_csv('Historia_Climatica.csv')
df["Fecha_Hora"] = pd.to_datetime(df["Fecha_Hora"], format="%m/%d/%y %H:%M")
df["Presion_hPa"] = df["Presion_hPa"] / 10
df.head()

,Fecha_Hora,Ciudad,Temperatura_C,Humedad_%,Vel_Viento_mps,Presion_hPa,Estado_Clima
0,2024-01-01 00:00:00,Madrid,27,79,184,993.5,Nublado
1,2024-01-01 01:00:00,Madrid,23,58,52,1012.5,Nublado
2,2024-01-01 02:00:00,Madrid,35,91,196,992.9,Nublado
3,2024-01-01 03:00:00,Madrid,11,96,114,1019.7,Nevado
4,2024-01-01 04:00:00,Madrid,34,62,147,1003.3,Tormentoso


In [42]:
# Convertir clases a números
le = LabelEncoder()
df["Estado_Clima_Cod"] = le.fit_transform(df["Estado_Clima"])

# Selección de características y etiquetas
X = df[["Temperatura_C", "Humedad_%", "Vel_Viento_mps", "Presion_hPa"]].values
y = df["Estado_Clima_Cod"].values

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [43]:
# Inicialización de pesos (Xavier, He, etc.)
def inicializacion_xavier(tamano):
  return np.random.randn(*tamano) * np.sqrt(1 / tamano[0])

def inicializacion_he(tamano):
  return np.random.randn(*tamano) * np.sqrt(2 / tamano[0])

# Ejemplo: inicialización de pesos para 4 entradas y 6 salidas
tamano = (4, len(np.unique(y)))
W_pesosX = inicializacion_xavier(tamano)
W_pesosH = inicializacion_he(tamano)

print("Pesos inicializados con Xavier:")
print(W_pesosX)

print("\nPesos inicializados con He:")
print(W_pesosH)


Pesos inicializados con Xavier:
[[-0.38910642  0.38825998 -0.33312117  1.12188504  0.11795086 -0.86051495]
 [-0.69220812  0.88999898 -0.24563739 -0.11683787  1.17549807  0.08855494]
 [ 0.08948395 -0.502815    0.4747012  -0.01086967  0.05511085 -0.10638563]
 [-0.20277206  0.02050023  0.98587433  0.13269855  0.50883426 -1.78150893]]

Pesos inicializados con He:
[[-0.03344275  0.02008107  0.31649231  0.94603021  0.70283111 -0.91484208]
 [ 0.40837277  0.18799491  0.66069781 -0.16385762  0.16127993  0.67138721]
 [-0.16048703 -0.19484161 -0.31219834  0.71321272 -1.36005872 -0.11808753]
 [-0.23605567 -0.84978643 -0.80681517 -0.18756673  0.0746559  -0.76656038]]


In [44]:
# Implementación de perceptrón simple (softmax + cross entropy)
class PerceptronSoftmax:
  def __init__(self, input_size, output_size, lr=0.01):
    self.W = W_pesosX
    self.b = np.zeros((1, output_size))
    self.lr = lr

  def softmax(self, z):
    e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return e_z / np.sum(e_z, axis=1, keepdims=True)

  def cross_entropy(self, y_pred, y_true):
    m = y_true.shape[0]
    log_likelihood = -np.log(y_pred[range(m), y_true])
    return np.sum(log_likelihood) / m

  def accuracy(self, y_pred, y_true):
    return np.mean(np.argmax(y_pred, axis=1) == y_true)

  def forward(self, X):
    z = np.dot(X, self.W) + self.b
    return self.softmax(z)

  def backward(self, X, y_true, y_pred):
    m = y_true.shape[0]
    grad = y_pred
    grad[range(m), y_true] -= 1
    grad /= m

    dW = np.dot(X.T, grad)
    db = np.sum(grad, axis=0, keepdims=True)

    self.W -= self.lr * dW
    self.b -= self.lr * db

  def train(self, X, y, epochs=100):
    for epoch in range(epochs):
      y_pred = self.forward(X)
      loss = self.cross_entropy(y_pred, y)
      acc = self.accuracy(y_pred, y)
      self.backward(X, y, y_pred)
      if epoch % 10 == 0:
        print(f"Época {epoch}: Pérdida={loss:.4f}, Precisión={acc:.4f}")

# Entrenar modelo de perceptrón simple
perceptron = PerceptronSoftmax(input_size=4, output_size=len(np.unique(y)), lr=0.1)
perceptron.train(X_train, y_train, epochs=100)

Época 0: Pérdida=2.4191, Precisión=0.1836
Época 10: Pérdida=2.2796, Precisión=0.1811
Época 20: Pérdida=2.1674, Precisión=0.1886
Época 30: Pérdida=2.0780, Precisión=0.1923
Época 40: Pérdida=2.0074, Precisión=0.1948
Época 50: Pérdida=1.9521, Precisión=0.1960
Época 60: Pérdida=1.9091, Precisión=0.1973
Época 70: Pérdida=1.8760, Precisión=0.2022
Época 80: Pérdida=1.8506, Precisión=0.2122
Época 90: Pérdida=1.8312, Precisión=0.2134


In [45]:
# Evaluación final
y_pred_test = perceptron.forward(X_test)
test_acc = perceptron.accuracy(y_pred_test, y_test)
test_acc

np.float64(0.14356435643564355)